In [7]:
# 크롤링
def Save_Book(url1):
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    import time

    driver = webdriver.Chrome()
    driver.get(url1)
    time.sleep(2)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # 책 링크 추출
    book_links = soup.select('a.prod_info')
    hrefs = [link.get('href').split('/')[-1] for link in book_links]

    books_list = []

    for i in hrefs:
        url = f'https://product.kyobobook.co.kr/detail/{i}'
        driver.get(url)
        time.sleep(2)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 기본 정보 추출
        try:
            title_text = soup.select_one('#contents .prod_title_box h1 span').get_text(strip=True) if soup.select_one('#contents .prod_title_box h1 span') else None
            author_text = soup.select_one('#scrollSpyProdInfo .product_person .title_wrap h3 a span').get_text(strip=True) if soup.select_one('#scrollSpyProdInfo .product_person .title_wrap h3 a span') else None
            price_text = soup.select_one('.prod_price_box span.price span').get_text(strip=True) if soup.select_one('.prod_price_box span.price span') else None
            total_review_text = soup.select_one('.klover_review_box .box_top .caption span.val').get_text(strip=True) if soup.select_one('.klover_review_box .box_top .caption span.val') else None
            review_num_text = soup.select_one('.tab_wrap .tab_list_wrap .sps_inner li:nth-child(3) a span span').get_text(strip=True) if soup.select_one('.tab_wrap .tab_list_wrap .sps_inner li:nth-child(3) a span span') else None
            isbn_text = soup.select_one('.basic_info .tbl_row_wrap td').get_text(strip=True) if soup.select_one('.basic_info .tbl_row_wrap td') else None
            page_text = soup.select_one('.basic_info tr:nth-child(3) td').get_text(strip=True) if soup.select_one('.basic_info tr:nth-child(3) td') else None
            content_text = soup.select_one('.book_intro .intro_bottom div:nth-child(2)').get_text(strip=True) if soup.select_one('.book_intro .intro_bottom div:nth-child(2)') else None
            category = soup.select_one('#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_book > ul > li').get_text(strip=True) if soup.select_one('#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_book > ul > li') else None
    
            # 작가 정보 추출
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
    
                button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[3]/div/div[2]/button')
                driver.execute_script("arguments[0].click();", button_author)
                time.sleep(2)
                author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_publish_review')
                author_info_text = author_info_section.text
    
            except Exception as e:
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
    
                    button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[3]/div[2]/div[2]/button')
                    driver.execute_script("arguments[0].click();", button_author)
                    time.sleep(2)
                    author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                    author_info_text = author_info_section.text
                except Exception as e:
                    try:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(2)
    
                        button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[3]/div[2]/div[2]/button')
    
                        driver.execute_script("arguments[0].click();", button_author)
                        time.sleep(2)
                        author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div.round_gray_box > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                        author_info_text = author_info_section.text
                    except Exception as e:
                        try:
                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            time.sleep(2)
    
                            button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[3]/div/div[2]/button')
    
                            driver.execute_script("arguments[0].click();", button_author)
                            time.sleep(2)
                            author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div > div.auto_overflow_contents > div > p')
                            author_info_text = author_info_section.text
                        except Exception as e:
                            try:
                                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                                time.sleep(2)
    
                                button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[3]/div[2]/div[2]/button')
    
                                driver.execute_script("arguments[0].click();", button_author)
                                time.sleep(2)
                                author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                                author_info_text = author_info_section.text
                            except Exception as e:
                                author_info_text = "작가 정보를 불러오는데 실패하였습니다."
    
    
            # 서평 추출
            try:
                button_review = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[2]/button')
                driver.execute_script("arguments[0].click();", button_review)
                time.sleep(2)
                review_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_publish_review')
                review_text = review_section.text
        
            except Exception as e:
                review_text = "서평 정보를 불러오는데 실패하였습니다."
    
    
                books_list.append({
                    
                    'title': title_text,
                    'author': author_text,
                    'price': price_text,
                    'rev_avg': total_review_text,
                    'rev_cou': review_num_text,
                    'ISBN': isbn_text,
                    'page': page_text,
                    'content': content_text,
                    'autho_info': author_info_text,
                    'book_rev': review_text,
                    'category': category})
    
    
        except Exception as e:
            print(f"책 정보를 가져오는 도중 오류 발생: {e}")

    df = pd.DataFrame(books_list)
    df['saleCmdtid']=hrefs
    return df

In [8]:
import pandas as pd

all_books_df = pd.DataFrame()

# 연도별로 반복
for year in range(19, 24): 
    for i in range(1, 11):
        url = f'https://product.kyobobook.co.kr/bestseller/total?period=004#?page={i}&per=20&period=004&ymw=20{year}&bsslBksClstCode=A'
        current_page_df = Save_Book(url)
        all_books_df = pd.concat([all_books_df, current_page_df], ignore_index=True)

In [9]:
# 결과 확
all_books_df.to_csv('all_book-info.csv')

,title,author,price,rev_avg,rev_cou,ISBN,page,content,autho_info,book_rev,category,saleCmdtid
0,여행의 이유,김영하,"12,150원",9.4,(3139),9788954655972,216쪽,여행의 감각을 일깨우는 소설가 김영하의 매혹적인 이야기 『여행의 이유』. 꽤 오래전...,"출판사 서평\n“나는 그 무엇보다 우선 작가였고, 그다음으로는 역시 여행자였다.”\...",서평 정보를 불러오는데 실패하였습니다.,국내도서>시/에세이>나라별 에세이>한국에세이,S000000780352
1,고요할수록 밝아지는 것들,혜민,"13,500원",9.7,None,9791187498384,272쪽,"자신을 돌아보는 계기를 마련해준 《멈추면, 비로소 보이는 것들》, 나와 타인과 세상...",따뜻한 소통법으로 많은 이들에게 위로와 용기를 전달하는 ‘동네 스님’. “혼자 힘들...,출판사 서평\n“가끔은 고요함 속에서 나를 만나는 시간을 선물하세요.”\n\n《멈추...,국내도서>시/에세이>나라별 에세이>한국에세이,S000001902595
2,나는 나로 살기로 했다,김수현,"12,420원",9.4,None,9791187119845,288쪽,“어른이 되어보니 세상은 냉담한 곳이었다.” 김수현은 책을 펴내면서 이런 말을 했다...,저자 김수현은 진지하지만 심각하지 않은 사람. 밝지만 가볍지 않은 사람. 미술학원에...,출판사 서평\n냉담한 현실에서 어른살이를 위한 To Do List\n\n비참해지려 ...,국내도서>시/에세이>나라별 에세이>한국에세이,S000001897363
3,철학은 어떻게 삶의 무기가 되는가,야마구치 슈,"14,400원",9.2,None,9791130620459,336쪽,한 치 앞을 알 수 없는 오늘을 사는 우리에게 가장 필요한 무기는 무엇일까? 누구보...,세계 1위 경영·인사 컨설팅펌 콘페리헤이그룹의 시니어 파트너.\n\n게이오 대학교 ...,출판사 서평\n★★★ 일본 아마존 인문·교양 베스트셀러 ★★★\n\n“왜 세계 최고...,국내도서>인문>철학>교양철학,S000001686647
4,봉제인형 살인사건,다니엘 콜,"13,500원",9.2,None,9788998274931,400쪽,2016년 4월 런던 도서전에서 가장 큰 주목을 받은 다니엘 콜의 데뷔작 『봉제인형...,출판사 서평\n영국 아마존 베스트셀러!\n2016년 런던 도서전 최고의 화제작\n영...,서평 정보를 불러오는데 실패하였습니다.,국내도서>소설>영미소설>미스터리/스릴러소설,S000001629834
...,...,...,...,...,...,...,...,...,...,...,...,...
805,착! 붙는 일본어 독학 첫걸음,일본어 공부기술연구소,"12,600원",9.6,(460),9788940291801,412쪽,None,저자 일본어 공부기술연구소는 국내 최초 일본어교육 전문기업 ㈜시사일본어사와 ㈜시사일...,출판사 서평\n▽ 착! 붙는 일본어 독학 첫걸음만의 차별화된 6가지 특징 ▽\n\n...,국내도서>외국어>일본어일반>일본어첫걸음,S000000641574
806,다정한 것이 살아남는다,브라이언 헤어,"19,800원",9.7,(398),9791197413025,396쪽,정세랑 작가가 쓴 추천의 말처럼 “어떤 책은 그 책이 가장 간절한 순간을 골라 찾아...,"Brian Hare\n듀크대학교에서 진화인류학, 심리학, 신경과학과 교수를 맡고 있...",출판사 서평\n★ 저자 친필 사인 인쇄본\n★ 국내 주요 매체 선정 2021년ㆍ20...,국내도서>과학>교양과학>교양유전>진화론/종의기원,S000001986381
807,이상한 과자 가게 전천당 17,히로시마 레이코,"11,700원",9.9,None,9791164065097,176쪽,베니코와의 대결로 연구소까지 잃게 된 로쿠조. 하지만 아직 끝나지 않았다!무너진 줄...,출판사 서평\n■〈전천당〉에 드리운 불행 그림자\n《이상한 과자 가게 전천당 17권...,서평 정보를 불러오는데 실패하였습니다.,국내도서>어린이(초등)>어린이문학>동화책>외국작가,S000201246670
808,나는 오래된 거리처럼 너를 사랑하고,진은영,"10,800원",9.7,(242),9788932040448,140쪽,2000년 『문학과사회』로 등단한 이후 시집 『일곱 개의 단어로 된 사전』(2003...,시인 진은영은 1970년 대전에서 태어나 이화여대 철학과를 졸업했다. 2000년 『...,출판사 서평\n낡고 익숙한 단어와 감각의 재배치로\n새롭게 태어나는 일상들-인식들\...,국내도서>시/에세이>한국시>현대시,S000061694107
